In [2]:
from pymatgen.core import Lattice, Structure, Site, IStructure
import numpy as np
import os
import ase.geometry
from pymatgen.io.vasp import Poscar
from ase.geometry.analysis import Analysis
import numpy as np
import os
from ase.io import read, write

## Helper Functions

In [3]:
def get_components(c0, c1):
    '''Parameters:
        - c0: list of Cartesian coordinates for neighboring atom
        - c1: list of Cartesian coordinates for atom to be replaced 
        Returns:
        -  x, y, z components of vector between both sets of coordinates'''
    return (c1[0]-c0[0], c1[1]-c0[1], c1[2]-c0[2])

def replace_distance(v, dist): # desired distance in Å
    '''Parameters: 
        - v: vector obtained by get_components()
        - desired distance between atoms in Ångstrom 
        Returns:
        - components of new vector '''
    return (v / np.linalg.norm(v)) * dist

def replace_coords(v, c0):
    '''Parameters:
        - v: new vector obtained with replace_distance()
        - c0: coordinates of neighboring atom
        Returns: 
        - updated coordinates of atom that will replace current atom'''
    return v + c0

## Modifying the Structure

In this thingy:
- just_the_inorganic(): saves just the inorganic components of a structure to a file
- replace_ligand_with_H(): truncates a site with a hydrogen by removing the attached ligand 
- remove_dupes(): removes duplicates within a certain cutoff radius
- selective_dynamics(): adds selective dynamics to a file

In [4]:
def just_the_inorganic(dest, mocha):
    '''saves just the inorganic components of a structure to a file
        - mocha: pymatgen.core.structure object
        - dest: where to save the file to'''
    organics_list = ['F', 'C', 'O', 'H', 'N']
    rm_list = []

    for a in range(len(mocha)):
        if str(mocha[a].species)[:-1] in organics_list and str(mocha[a].species)[:-1] not in rm_list:
            rm_list.append(str(mocha[a].species)[:-1])
    
    for r in rm_list:
        mocha.remove_species(species=r)
    
    if not os.path.exists(f"{dest}/structure_analysis"):
        os.mkdir(f"{dest}/structure_analysis")
    mocha.to(filename = f"{dest}/structure_analysis/inorganic.cif")
    print(f"Saved to {dest}/structure_analysis/inorganic.cif")


def replace_ligand_with_H(dest, mocha, removals, temp_site):
    '''Replaces the ligand at the chalcogen site with a H. 
        Purpose is to isolate the inorganic interaction of the 
        MOCha at the band gap. 
        dest - where to store final file
        mocha - Pymatgen Structure object
        removals - list of strings in which each string are the species
        that make up the ligand
        temp_site - site that will be replaced with a hydrogen at the
        appropriate distance from the calchogen'''

    chalcogens = ['Se', 'Te', 'S']

    # distance in Angstrom between the given chalcogen and a hydrogen
    chalcogen_hydrogen_distances = {'Se': 1.46, 'S': 1.3, 'Te': 1.68}

    # removing atoms that make up ligand
    for r in removals:
        mocha.remove_species(species=r)

    for a in range(len(mocha)):
        if str(mocha[a].species) == f"{temp_site}1":    
            print(str(mocha[a].species))
            coordz = mocha[a].coords
            neighbors = mocha.get_neighbors(site = mocha[a], r = 2.8)
            for n in neighbors:
                print(n.species)
                if str(n.species)[:-1] in chalcogens:
                    print("your mom")
                    curr_vector = get_components(n.coords, mocha[a].coords) # current distance in Å
                    new_vector = replace_distance(curr_vector, chalcogen_hydrogen_distances[str(n.species)[:-1]]) # desired length in Å
    
                    # replace atoms with temp species not already in molecule
                    mocha.replace(idx=a, species='H', coords=replace_coords(new_vector, n.coords), coords_are_cartesian=True)

    mocha.to(filename = f"{dest}/inorganic-with-H.cif") # write to file
    print(f"file written to {dest}/inorganic-with-H.cif")   

def remove_dupes(structure):
    '''Removes duplicates from a structure
        - structure: ase Atoms object'''
    
    print(f"Total: {len(structure)}")
    new_struct = ase.geometry.get_duplicate_atoms(structure, cutoff=0.2, delete=True)
    new_struct = ase.geometry.get_duplicate_atoms(structure, cutoff=0.2, delete=False)
    print(f"Total with duplicates removed: {len(new_struct)}")

    return structure 

def selective_dynamics(yourmom, filename, species):
    '''Adds selective dynamics to desired species
        - yourmom: pymatgen.core.structure object
        - filename: str filename to save Structure object
        - species: list of strings; species to add selective dynamics'''
    
    select_dicks = []

    for ass in range(len(yourmom)):
        if str(yourmom[ass].species)[:-1] in species:
            select_dicks.append([True, True, True])
        else:
            select_dicks.append([False, False, False])
    # convert to poscar
    pos = Poscar(yourmom, selective_dynamics=select_dicks)

    pos.write_file(filename)
    print((f"File written to {filename}"))

def remove_species(dest, filename, mocha, species):
    '''saves just the inorganic components of a structure to a file
        - mocha: pymatgen.core.structure object
        - dest: where to save the file to
        - species: string list of species to remove from Structure'''
    
    for s in species:
        mocha.remove_species(species=s)
    
    poscar = Poscar(mocha)
    poscar.write_file(f"{dest}/{filename}")
    print(f"Saved to {dest}/{filename}")

def find_unit_cell(path, filename, supercell):
    unit_cell = supercell.get_primitive_structure()
    poscar = Poscar(unit_cell)
    poscar.write_file(f"{path}/{filename}_unitcell.vasp")
    print(f"Unit cell saved to {path}/{filename}_unitcell.vasp")

## Get the unit cell from a supercell structure

In [7]:
# filename = "CuS-4MeO_ORANGE"

# supercell_path = f"/Users/adrianaladera/Desktop/MIT/research/mochas/ALL_STRUCTURES/fluorescence_1D_mochas/1D_fluorescent/{filename}"
# structure = Structure.from_file(f"{supercell_path}/{filename}.vasp")

path = "/Users/adrianaladera/Desktop/MIT/research/mochas/ALL_STRUCTURES/fluorescence_1D_mochas/new_mochas/"
structure = Structure.from_file(f"{path}/AuSEtPh_high-temp_ORANGE.cif")
find_unit_cell(path, "AuSEtPh_high-temp_ORANGE", structure)

Unit cell saved to /Users/adrianaladera/Desktop/MIT/research/mochas/ALL_STRUCTURES/fluorescence_1D_mochas/new_mochas//AuSEtPh_high-temp_ORANGE_unitcell.vasp


## Remove selected species from a structure

In [8]:
mocha = "t_dimethyl_1D"
path = f"/Users/adrianaladera/Desktop/MIT/research/mochas/ALL_STRUCTURES/3-amino_AgS-anilines/dimethyl_1D/{mocha}"

structure = Structure.from_file(f"{path}/{mocha}_skeleton.vasp")
remove_species(path, f"{mocha}.vasp", structure, ["H"])

Saved to /Users/adrianaladera/Desktop/MIT/research/mochas/ALL_STRUCTURES/3-amino_AgS-anilines/dimethyl_1D/tert-butyl_dimethyl_1D/tert-butyl_dimethyl_1D.vasp


## Get selective dynamics on certain species within a structure to run in VASP

In [9]:
mocha = "2-methoxy_dimethyl_1D"
path = f"/Users/adrianaladera/Desktop/MIT/research/mochas/ALL_STRUCTURES/3-amino_AgS-anilines/dimethyl_1D/{mocha}"
structure = Structure.from_file(f"{path}/{mocha}_FINAL.vasp")

selective_dynamics(structure, f"{path}/{mocha}_FINAL_sel_dyn.vasp", ["H", "C", "N"])

File written to /Users/adrianaladera/Desktop/MIT/research/mochas/ALL_STRUCTURES/3-amino_AgS-anilines/dimethyl_1D/2-methoxy_dimethyl_1D/2-methoxy_dimethyl_1D_FINAL_sel_dyn.vasp


In [19]:
path = "/Users/adrianaladera/Desktop/yourmom/"
filename = "temp.vasp"
structure = Structure.from_file(f"{path}balls.vasp")
select_dicks = []
for i in range(len(structure)):
    select_dicks.append([True, True, True])

new_struct = Structure(lattice=structure.lattice, species=structure.species, coords=structure.frac_coords)

pos = Poscar(new_struct)
pos.write_file(f"{path}{filename}")


In [5]:
# path to MOChA file that you want to modify
mocha = "1D-AgSe-benzene_3methoxy-I"
path = f"/Users/adrianaladera/Desktop/MIT/research/mochas/VASP_calculations/varying_inorganics/AgS-benzene/1D-AgS-benzene_3methoxy-I/3ISIF/scf"
dest = path[:-4]
# mocha = "CONTCAR"

structure = Structure.from_file(f"{path}/POSCAR".format(path))
just_the_inorganic(f"{path}", structure)

Saved to /Users/adrianaladera/Desktop/MIT/research/mochas/VASP_calculations/varying_inorganics/AgS-benzene/1D-AgS-benzene_3methoxy-I/3ISIF/scf/structure_analysis/inorganic.cif


In [24]:
root = "/Users/adrianaladera/Desktop/MIT/research/mochas/VASP_calculations/sugar_mochas/galac_rtr_dehydrated/2ISIF/" 

stuff = root.split("/")

print(stuff[-2], stuff[-3])

2ISIF galac_rtr_dehydrated
